## AWS NOTEBOOK
This notebook is used to create the AWS resources that will be used in the sparkify ETL project.
Fill the dwh2.cfg file before running the cells in this notebook

In [ ]:
import pandas as pd
import boto3
import json
import configparser
import numpy

In [ ]:
config =configparser.ConfigParser()
config.read_file(open('dwh2.cfg'))

KEY = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')
DB_NAME = config.get('CLUSTER','DB_NAME')
DB_USER = config.get('CLUSTER','DB_USER')
DB_PASSWORD = config.get('CLUSTER','DB_PASSWORD')
DB_PORT= config.get('CLUSTER','DB_PORT')
ARN = config.get('IAM_ROLE','ARN')
IAM_ROLE_NAME = config.get('IAM_ROLE','IAM_ROLE_NAME')
CLUSTER_NODE_TYPE=config.get('CLUSTER','CLUSTER_NODE_TYPE')
CLUSTER_TYPE=config.get('CLUSTER','CLUSTER_TYPE')
CLUSTER_IDENTIFIER=config.get('CLUSTER','CLUSTER_IDENTIFIER')
CLUSTER_NUM_NODE=config.get('CLUSTER','CLUSTER_NUM_NODES')




pd.DataFrame({"Param":
    ["CLUSTER_NODE_TYPE","CLUSTER_TYPE",
    "CLUSTER_IDENTIFIER","CLUSTER_NUM_NODE",
    "DB_NAME","DB_USER","DB_PASSWORD","DB_PORT",
    "ARN","IAM_ROLE_NAME","KEY","SECRET"],
    "Values":[CLUSTER_NODE_TYPE,CLUSTER_TYPE,
            CLUSTER_IDENTIFIER, CLUSTER_NUM_NODE,
              DB_NAME,DB_USER,DB_PASSWORD,DB_PORT,
              ARN,IAM_ROLE_NAME,KEY,SECRET]
              })

In [ ]:
s3= boto3.resource('s3',
                    region_name='us-west-2',
                    aws_access_key_id=KEY,
                    aws_secret_access_key = SECRET
                    )

iam = boto3.client('iam',
                   aws_access_key_id=KEY,
                   aws_secret_access_key = SECRET,
                   region_name='us-west-2'
                   )

redshift = boto3.client('redshift',
                        region_name='us-west-2',
                        aws_access_key_id=KEY,
                        aws_secret_access_key = SECRET,
                        )

Attach the role created with AmazonS3ReadOnly access, remember to specify the name in dwg2.cfg

In [ ]:
#Attach  IAMROLE

from botocore.exceptions import ClientError

# try:
#     dwhRole = iam.create_role(
#         Path='/',
#         RoleName=IAM_ROLE_NAME,
#         Description="Allows Redshift cluster to call AWS services on your behalf.",
#         AssumeRolePolicyDocument=json.dumps(
#             {
#                 'Statement':[{
#                     'Action':'sts:AssumeRole',
#                     'Effect':'Allow',
#                     'Principal':{'Service':'redshift.amazonaws.com'}
#                 }],
#                 'Version':'2012-10-17'
#             }
#         )
#     )
# except Exception as e:
#     print(e)
    
print("Attaching policy")

iam.attach_role_policy(RoleName=IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                       )['ResponseMetadata']['HTTPStatusCode']
                         
                       
print("Getting the IAM role ARN") 

roleArn= iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

    

#### Creating Redshift Cluster

In [ ]:
try: 
    response = redshift.create_cluster(
        ClusterType=CLUSTER_TYPE,
        NodeType=CLUSTER_NODE_TYPE,
        NumberOfNodes=int(CLUSTER_NUM_NODE),        
        DBName=DB_NAME,
        ClusterIdentifier=CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,        
        IamRoles=[ARN]
        
    )
except Exception as e:
    print(e)

Wait a little while before running the next cell. if the output of cluster status is **creating** wait a bit again and try in 5 mins.

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth',-1)
    keysToShow=["ClusterIdentifier","NodeType","ClusterStatus",
                "MasterUserName","DBName","Endpoint","NumberOfNodes","VpcId"]
    x =[(k,v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key","Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ",DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

Keep the ARN with the DWH_ENDPOINT and DWH_ROLE_ARN and input the value for host and ARN respectively.

#### Open an incoming TCP port to access the cluster ednpoint

In [ ]:
try:
    vpc=ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DB_PORT),
        ToPort=int(DB_PORT)
    )
except Exception as e:
    print(e)

### Delete AWS Resources
When done  with all the scripts, clean up you resources to avoid being charged

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=IAM_ROLE_NAME)
#### CAREFUL!!